In [56]:
import cv2
import matplotlib.pyplot as plt
import re
import collections
from keras.applications.resnet50 import ResNet50,preprocess_input
from keras.optimizers import Adam
from keras.layers import *
from keras.models import Model
from keras.models import Model,load_model

In [57]:
import numpy as np
from keras.preprocessing import image
import pickle
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Input,Dropout,Dense




In [58]:
model = ResNet50(weights='imagenet',input_shape=(224,224,3))
model_new = Model(model.input,model.layers[-2].output)


A local file was found, but it seems to be incomplete or outdated because the md5 file hash does not match the original value of a7b3fe01876f51b976af0dea6bc144eb so we will re-download the data.
102858752/102853048 [==============================] - 116s 1us/step


In [59]:
def preprocess_img(img):
    img = image.load_img(img,target_size=(224,224))
    img = image.img_to_array(img)
    img = np.expand_dims(img,axis=0) # changes (224,224,3)-->(1,224,224,3)
    #Normalization
    img = preprocess_input(img)
    return img

In [60]:
def encode_image(img):
    img = preprocess_img(img)
    feature_vector = model_new.predict(img)
    
    feature_vector = feature_vector.reshape((-1,))
    return feature_vector

In [61]:
final_model = load_model('./model_weights/model_9.h5')


In [62]:

import pickle
with open(".\model_weights\word_to_idx.pkl","rb") as w2i:
    word_to_idx = pickle.load(w2i)
with open(".\model_weights\idx_to_word.pkl","rb") as i2w:
    idx_to_word = pickle.load(i2w)

In [63]:
max_len = 35
def predict_captions(photo):
    in_text = "startseq"
    for i in range(max_len):
        sequence = [word_to_idx[w] for w in in_text.split() if w in word_to_idx]
        sequence = pad_sequences([sequence],maxlen=max_len,padding='post')
        
        ypred = final_model.predict([photo,sequence])
        ypred = ypred.argmax() #Word with max probability (We can also add some randomness)
        word = idx_to_word[ypred]
        in_text += ' '+word
        
        if word == 'endseq':
            break
    
    final_caption = in_text.split()[1:-1]
    final_caption = ' '.join(final_caption)
    
    return final_caption

In [64]:
def predict_caption(photo):
    encoded_image = encode_image(photo)
    photo_2048 = encoded_image.reshape((1,2048))
    #img = plt.imread(photo)
    caption = predict_captions(photo_2048)
    return caption
    #print(caption)
    #plt.imshow(img)
    #plt.axis('off')
    #plt.show()

In [67]:
caption = predict_caption('he.jpg')

In [68]:
print(caption)

man is sitting on bench with his feet on his bike
